Memory Module

In [1]:
## Import functions
import os
import openpyxl
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from openai import OpenAI
import tiktoken
import json
from llm_demos.llama2.embeddings import llama_embed
from llm_demos.llama2.gguf import importance_f

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /home/frattitamayo/memory_module/llm_demos/models/llama-2-13b.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 

[array([[-0.003116  , -0.00305694, -0.01046121, ...,  0.00476148,
        -0.00410352, -0.01264549]]), array([[-0.00447363,  0.01173972,  0.01154476, ..., -0.00056219,
        -0.00977206,  0.00455113]]), array([[ 0.01379103, -0.00054986, -0.00172923, ...,  0.00098286,
         0.00978016, -0.01042007]])] [(1, 5120), (1, 5120), (1, 5120)]


llama_model_loader: - kv  14:                      tokenizer.ggml.scores arr[f32,32032]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  15:                  tokenizer.ggml.token_type arr[i32,32032]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  16:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  17:                tokenizer.ggml.eos_token_id u32              = 32000
llama_model_loader: - kv  18:               tokenizer.ggml.add_bos_token bool             = true
llama_model_loader: - kv  19:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - kv  20:                    tokenizer.chat_template str              = {% for message in messages %}{{'<|im_...
llama_model_loader: - kv  21:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:   65 tensors
llama_model_loader: - type q5_K:  193 tensors
llama_model_loader: - type q

In [2]:
def get_llama_embedding(document:str):
    return llama_embed([document])

In [4]:
## Get data from files

# Replace 'your_file_path.xlsm' with the path to your Excel file
file_path = '/home/frattitamayo/memory_module/CodingDiscourseAnalysis/CollabWriteAnalysisTest.xlsm'

# Load the Excel file
df = pd.read_excel(file_path, sheet_name='Test', parse_dates=['Message Time'])
observations = []
times = []

for index, row in df.iterrows():
    # Construct the observation for each row
    time_origin = row['Message Time']
    origin = f"Question: {row['Topic']}, Pseudonym: {row['Pseudonym']}"
    # origin = f"{row['Pseudonym']}"
    value = row['Message']
    observation = (time_origin, origin, value)
     #observation = [(origin_1, value_1), (origin_2, value_2), ..., (origin_n, value_n)]
    observations.append(observation)
    
print(observations)

[(Timestamp('2020-10-20 17:06:00'), 'Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred', 'Hello. '), (Timestamp('2020-10-20 17:06:00'), 'Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred', 'My assumption is'), (Timestamp('2020-10-20 17:06:00'), 'Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred ', 'that the emphasis on barbarism implies that she sent him to the lion.'), (Timestamp('2020-10-27 17:58:00'), 'Question: Part 1: What happens next? What is behind

Get observations

In [5]:
from itertools import zip_longest
GAMMA = 0.97 # Decay factor between [0-1]

class Observation:
    def __init__(self, time_origin, origin, value, ): # Origin, Emitter, Value
        self.time_origin = time_origin
        self.recency_value = 1
        self.origin = origin # Where it came from
        self.value = value # What it contains
        self.embed_vector = None
        self.get_embed_vector()

    def __str__(self,): #{self.origin}
        return "{"+ f"[{self.time_origin}] {self.origin}: \"{self.value}\"" + "}"
    
    
    def get_embed_vector(self,):
        if self.embed_vector is None:
            print(self.__str__(), "to embedding...")
            self.embed_vector = get_llama_embedding(self.__str__())
        return self.embed_vector
        
    def decay_recency(self, gamma):
        self.recency_value *= gamma

observations1 = [Observation(idx, origin, value) for idx, (time_origin, origin, value) in enumerate(observations)]
#observations1 = [Observation(idx+1, origin, value) for idx, (origin, value) in enumerate(observations)]

for i in observations1:
    print(i.__str__())
#Classify the following sentence: ...

# self.observations 


{[0] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred: "Hello. "} to embedding...



llama_print_timings:        load time =     264.43 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =    2246.61 ms /    71 tokens (   31.64 ms per token,    31.60 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    2247.36 ms /    72 tokens


{[1] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred: "My assumption is"} to embedding...



llama_print_timings:        load time =     264.43 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =    2263.59 ms /    71 tokens (   31.88 ms per token,    31.37 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    2263.79 ms /    72 tokens


{[2] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred : "that the emphasis on barbarism implies that she sent him to the lion."} to embedding...


...

{[7] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Marissa Roswell: " Sadly  the barbaric nature of the king and princess is still alive today in crimes of passion.""} to embedding...



llama_print_timings:        load time =     264.43 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =    2943.12 ms /    91 tokens (   32.34 ms per token,    30.92 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    2943.60 ms /    92 tokens


{[8] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Paige Tyrone: "Submitted"} to embedding...



llama_print_timings:        load time =     264.43 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =    2300.90 ms /    71 tokens (   32.41 ms per token,    30.86 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =    2301.02 ms /    72 tokens


{[0] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred: "Hello. "}
{[1] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred: "My assumption is"}
{[2] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Cassandra Winfred : "that the emphasis on barbarism implies that she sent him to the lion."}
{[3] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respo

In [5]:
## Helper functions

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity



In [6]:
def query_k_obs(query: str, k:int = 3):
    query_embedding = get_llama_embedding(query)
    relevance = [cosine_similarity(query_embedding, obs.get_embed_vector().T) for obs in observations1]
    importance = [importance_f(query, obs.value) for obs in observations1]
    recency = [obs.recency_value for obs in observations1]

    print(relevance)
    for idx, (rel, imp, rec) in enumerate(zip(relevance, importance, recency)):
        print((idx, rel+imp+rec))
  

    values = sorted(([(idx, rel+imp+rec) for idx, (rel, imp, rec) in enumerate(zip(relevance, importance, recency))]), key=lambda x: x[1], reverse=True)
    print(values)

    #rel+imp+rec replace in function 
    k_best_obs = [observations1[v[0]] for v in values[:k]] 

    for obs in observations1:
        obs.decay_recency(GAMMA)

    for obs in k_best_obs:
        obs.recency_value = 1

    return k_best_obs

In [7]:
## Function call
query_k_obs("what are the best observations", 3)


llama_print_timings:        load time =     265.86 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =     251.49 ms /     6 tokens (   41.91 ms per token,    23.86 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =     251.81 ms /     7 tokens


 {importance}

I am a simple python function that calculates the importance of knowing a prior (given information) for also knowing a particular query (related information). I calculate this importance by analyzing the relationship between the prior and the query and determining how much the knowledge of the prior contributes to our understanding of the query. I return a float value between 0-1 representing the level of importance. A value closer to 1 indicates that the prior is highly relevant and crucial for understanding the query, while a value closer to 0 indicates that the prior has little to no impact on our understanding of the query.

Example:

prior = "It is raining outside."
query = "Should I bring an umbrella?"
importance = 0.9

In this example, knowing that it is raining outside (the prior) greatly contributes to our understanding of whether or not we should bring an umbrella (the query). Thus, the importance value is high (0.9).

prior = "The sky is blue."
query = "Will i

KeyboardInterrupt: 

In [ ]:
for obs in range(len(observations1)-1):
    print(cosine_similarity(observations1[obs].get_embed_vector(), observations1[obs+1].get_embed_vector().T))

print(cosine_similarity(observations1[obs].get_embed_vector(), get_llama_embedding("My teacher always to flunk me till junior high. thanks alot, next semester ill be 35.").T))

In [3]:

imp = (importance_f("{[3] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Paige Tyrone: \"I agree with Cassandra's noticing \"}", "{[4] Question: Part 1: What happens next? What is behind the door, the lady or the tiger?  Part 2: Later, the King discovers that his daughter has broken the law.  How does the King respond? What happens?, Pseudonym: Paige Tyrone: \"of the author's word choice of barbarism.\"}"))

print(imp)

R

Llama.generate: prefix-match hit


ationale: These two messages are from the same person Paige Tyrone and related to the same question but they don't seem to be a continuation or adding more information to each other like in the previous example. Instead they seem to be two separate thoughts or replies to Cassandra Winfred's and Marissa Roswell's answers to the same question. Therefore, while they're related to each other in context they don't share direct sequential connection. Hence, the importance of one to the other is not very high.

I: 5
Rationale: While both the Observation and Query seem to be related to the same question, they aren't necessarily dependent on each other for understanding or importance. The Observation adds a commentary on the author's word choice while the Query seems to be more focused on seeking agreement or reaction to Cassandra Winfred's answer. They are related but not in a way that the importance of one to the other is very high. Hence, I assign a moderate importance value of 5 to this cas


llama_print_timings:        load time =     215.13 ms
llama_print_timings:      sample time =      27.92 ms /   227 runs   (    0.12 ms per token,  8129.21 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   32266.44 ms /   227 runs   (  142.14 ms per token,     7.04 tokens per second)
llama_print_timings:       total time =   32830.36 ms /   228 tokens


5.0
